In [5]:
import os

# get URLs
urls = [url.strip() for url in open("cad_urls.txt").readlines()]

for i,url in enumerate(urls):
    split_url = url.split("/")
    dir = split_url[-1]
    urls[i] = {"show_url":url,"dir":dir}

urls[0]

{'show_url': 'https://www.contemporaryartlibrary.org/project/michael-morley-at-100-bell-towers-montreal-29047',
 'dir': 'michael-morley-at-100-bell-towers-montreal-29047'}

In [68]:
# some errors in this code. worked fairly well, some of the links didn't populate the dictionary as desired, had to do some minor tweeking to get it right. Sorry future people!

import requests
import json
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

ix = 6773

for url in tqdm(urls):
    response = requests.get(url["show_url"])
    
    try:
        response.raise_for_status()  # Raise an exception for HTTP errors
        show_html = response.text
        soup = BeautifulSoup(show_html, 'html.parser')


        script_tag = soup.find('script', id='__NEXT_DATA__')

        if script_tag:
            json_data = json.loads(script_tag.contents[0])
            
            # Access the data structure safely with try-except blocks
            try:
                show_images = json_data['props']['pageProps']['projectObject']['images']
            except KeyError:
                show_images = []  # Set to an empty list if 'projectObject' is missing
            
            img_urls = []
            for img in show_images:
                img_urls.append(img.get("large"))  # Use .get() to handle missing keys
            
            urls[ix]["img_urls"] = img_urls

            # Find the <ul> element with class "Caption_docs__GcbVl"
            ul_docs = soup.find('ul', class_='Caption_docs__GcbVl')
            
            if ul_docs:
                # Find all <a> tags within the <ul> element
                a_tags = ul_docs.find_all('a')
                for a_tag in a_tags:
                    if a_tag.text == 'Press Release':
                        press_release_url = a_tag.get('href')
                        #print(press_release_url)
                        urls[ix]["press_release_url"] = press_release_url
                        break  # Stop searching after finding the first "Press Release" link
            try:
                url["venue"] = url["show_url"].split('/')[-1].split('-at-',1)[1].rsplit('-',1)[0].replace('-',' ')
            except:
                url["venue"] = None
            
            try:
                artist_name = soup.find('span', class_='Caption_artist__MVASe').text
                show_title_element = soup.find(class_='Caption_title__2iQzQ')
                show_title = show_title_element.i.span.text if show_title_element and show_title_element.i and show_title_element.i.span else None
                
                url["artist"] = artist_name
                url["show_title"] = show_title
            except:
                url["artist"] = None
                url["show_title"] = None
                print(f"url: {url['show_url']} |")
                

        else:
            print(f"Script tag {ix} with id '__NEXT_DATA__' not found.")
    except requests.exceptions.RequestException as e:
        print(f"Error retrieving data from {url}: {e}")
    
    ix += 1

json_filename = "data.json"

with open(json_filename, "w") as json_file:
    json.dump(urls, json_file)


  0%|          | 0/9487 [00:00<?, ?it/s]

url: https://www.contemporaryartlibrary.org/project/argento-e-oro-at-htsu-how-to-show-up-amsterdam-26714 |
url: https://www.contemporaryartlibrary.org/project/ain-bailey-at-htsu-how-to-show-up-amsterdam-26712 |
url: https://www.contemporaryartlibrary.org/project/holly-white-at-htsu-how-to-show-up-amsterdam-26706 |
url: https://www.contemporaryartlibrary.org/project/landscape-modern-oil-painting-canvas-painting-abstract-oil-at-gl-holtegaard-11492 |
url: https://www.contemporaryartlibrary.org/project/daisy-chain-at-joan-los-angeles-4587 |
url: https://www.contemporaryartlibrary.org/project/olivia-mole-at-joan-los-angeles-4581 |
url: https://www.contemporaryartlibrary.org/project/lea-cetera-at-joan-4507 |
url: https://www.contemporaryartlibrary.org/project/dispersion-taxonomy-at-joan-los-angeles-4586 |
url: https://www.contemporaryartlibrary.org/project/keren-cytter-at-kolnischer-kunstverein-cologne-1364 |
url: https://www.contemporaryartlibrary.org/project/_________-at-kristina-kite-gall

IndexError: list index out of range

In [115]:
import json

with open("data.json","r") as json_file:
    urls = json.load(json_file)
    
len(urls)

16256

In [ ]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import json

# Function to download a single image
def download_image(img_url, img_path):
    try:
        response = requests.get(img_url, stream=True)
        response.raise_for_status()

        # Check if the image file already exists, skip if it does
        if not os.path.exists(img_path):
            with open(img_path, 'wb') as img_file:
                for chunk in response.iter_content(chunk_size=8192):
                    img_file.write(chunk)
    except Exception as e:
        print(f"Failed to download {img_url}: {e}")

# Create the 'img' directory if it doesn't exist
img_directory = 'img'
os.makedirs(img_directory, exist_ok=True)

# Loop through the list of dictionaries
#for item in tqdm(urls[:10]):
    dir_name = item['dir']
    img_urls = item['img_urls']

    # Create a subdirectory within 'img' based on 'dir' value
    subdirectory_path = os.path.join(img_directory, dir_name)
    os.makedirs(subdirectory_path, exist_ok=True)

    # Create a ThreadPoolExecutor with a maximum of 5 threads (adjust as needed)
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = []

        # Download and save images in parallel
        for img_url in img_urls:
            img_filename = os.path.basename(img_url)
            img_path = os.path.join(subdirectory_path, img_filename)

            futures.append(executor.submit(download_image, img_url, img_path))

        # Wait for all download tasks to complete
        for future in futures:
            future.result()

    # Download and save the press release
    press_release_url = item.get('press_release_url')
    if press_release_url:
        press_release_filename = f"pr_{dir_name}.pdf"
        press_release_path = os.path.join(subdirectory_path, press_release_filename)

        # Check if the press release file already exists, skip if it does
        if not os.path.exists(press_release_path):
            download_image(press_release_url, press_release_path)
